# Data analysis: SCRUM KPIs

## Data import

In [4]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Clean data

In [5]:
# Rename columns for display
def extract_KPI_name(s):
    match = re.search(r'\[(.*?)\]', s)
    return match.group(0) if match else None

df = pd.read_csv('./survey-results.csv');
experience_column = 'How many years of expirience do you have with SCRUM?'
role_columns = [col for col in df.columns if 'Which SCRUM roles have you primarily taken on?' in col and not 'Other' in col]
other_role_column = [col for col in df.columns if 'Which SCRUM roles have you primarily taken on?' in col and 'Other' in col][0]
kpi_columns = [col for col in df.columns if 'How helpful do you find the following KPIs for SCRUM purposes?' in col and 'Scale 1' in col]
use_kpi_columns = [col for col in df.columns if 'How helpful do you find the following KPIs for SCRUM purposes?' in col and 'Scale 2' in col]

# Only people that have experience with SCRUM are relevant to this survey
df = df[(df['Do you have experience with SCRUM?'] == 'Yes')]

yes_no_replacement = {'Yes': 1, 'No' : 0}

# Clean and replace everything with numerical numbers that is relevant for aggregation computations
df['Do you have experience with SCRUM?'] = df['Do you have experience with SCRUM?'].map({'Yes': 1})
df[role_columns] = df[role_columns].replace(yes_no_replacement)
df[kpi_columns] = df[kpi_columns].replace({'1 - Not helpful at all': 1, '5 - Very helpful' : 5})
df[use_kpi_columns] = df[use_kpi_columns].replace(yes_no_replacement)


In [6]:
# Correlations

rename_dict = {col: extract_KPI_name(col) for col in kpi_columns}
corr_df = df.copy()
corr_matr = corr_df[[*role_columns, *kpi_columns]].corr()

threshold = 0.75

high_correlation = (corr_matr.where(np.triu(np.ones(corr_matr.shape), k=1).astype(bool))
                    .stack()
                    .reset_index())

high_correlation.columns = ['Variable 1', 'Variable 2', 'Correlation Coefficient']

high_correlation = high_correlation[abs(high_correlation['Correlation Coefficient']) > threshold]

for index, row in high_correlation.iterrows():
    print(f"Variable 1: {row['Variable 1']}, Variable 2: {row['Variable 2']}, Correlation: {row['Correlation Coefficient']}\n")

corr_matr

Variable 1: How helpful do you find the following KPIs for SCRUM purposes? [Number of planned Tickets][Scale 1], Variable 2: How helpful do you find the following KPIs for SCRUM purposes? [Number of done Tickets][Scale 1], Correlation: 0.9056263923284888

Variable 1: How helpful do you find the following KPIs for SCRUM purposes? [Blocker Tickets (Number of 'emergency' tickets that got added during a Sprint)][Scale 1], Variable 2: How helpful do you find the following KPIs for SCRUM purposes? [Blocker Tickets done (Number of 'emergency' tickets that got done in the Sprint)][Scale 1], Correlation: 0.7600239746492246



,Which SCRUM roles have you primarily taken on? [Product Owner],Which SCRUM roles have you primarily taken on? [SCRUM team member],Which SCRUM roles have you primarily taken on? [SCRUM master],How helpful do you find the following KPIs for SCRUM purposes? [Capacity (How many Story Points can a Team theoretically do in a Sprint?)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Number of planned Tickets][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Number of done Tickets][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Commitment (Story Points planned for a Sprint)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Overplanning ratio (Planned Story Points / Capacity)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Amount of done Story Points][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Velocity (Planned Story Points / Done Story Points)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Blocker Tickets (Number of 'emergency' tickets that got added during a Sprint)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Blocker Tickets done (Number of 'emergency' tickets that got done in the Sprint)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Additional Tickets (Tickets that got added unexpectedly during the Sprint)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Open Story points (How many story points are not done at the end of a Sprint and what state are they in?)][Scale 1],How helpful do you find the following KPIs for SCRUM purposes? [Removed Tickets (Tickets that got removed during a Sprint)][Scale 1]
Which SCRUM roles have you primarily taken on? [Product Owner],1.000000,-0.576195,0.070014,0.191182,-0.079120,-0.112473,-0.027253,-0.021499,-0.014754,0.048328,-0.146377,-0.124423,-0.083272,-0.059923,0.225569
Which SCRUM roles have you primarily taken on? [SCRUM team member],-0.576195,1.000000,-0.233955,-0.189558,-0.136433,-0.165809,-0.171420,-0.011973,-0.040782,-0.131938,0.009398,0.077677,0.012175,-0.034690,-0.170421
Which SCRUM roles have you primarily taken on? [SCRUM master],0.070014,-0.233955,1.000000,-0.018124,-0.087323,-0.120483,-0.077850,-0.092120,-0.118534,0.076696,-0.215372,-0.134122,-0.122868,-0.363450,-0.047225
How helpful do you find the following KPIs for SCRUM purposes? [Capacity (How many Story Points can a Team theoretically do in a Sprint?)][Scale 1],0.191182,-0.189558,-0.018124,1.000000,-0.020791,0.046098,0.698408,0.379978,0.653233,0.639410,0.146296,0.076637,0.142517,0.351664,0.107144
How helpful do you find the following KPIs for SCRUM purposes? [Number of planned Tickets][Scale 1],-0.079120,-0.136433,-0.087323,-0.020791,1.000000,0.905626,0.159955,-0.046793,-0.053107,-0.118189,0.420931,0.356526,0.112342,0.143030,0.159023
How helpful do you find the following KPIs for SCRUM purposes? [Number of done Tickets][Scale 1],-0.112473,-0.165809,-0.120483,0.046098,0.905626,1.000000,0.234490,-0.024664,0.042315,-0.046203,0.390285,0.278301,0.142114,0.122422,0.208625
How helpful do you find the following KPIs for SCRUM purposes? [Commitment (Story Points planned for a Sprint)][Scale 1],-0.027253,-0.171420,-0.077850,0.698408,0.159955,0.234490,1.000000,0.496032,0.638260,0.507519,0.370094,0.204477,0.275480,0.508846,0.147058
How helpful do you find the following KPIs for SCRUM purposes? [Overplanning ratio (Planned Story Points / Capacity)][Scale 1],-0.021499,-0.011973,-0.092120,0.379978,-0.046793,-0.024664,0.496032,1.000000,0.536532,0.694754,0.297602,0.271417,0.581778,0.481216,0.356621
How helpful do you find the following KPIs for SCRUM purposes? [Amount of done Story Points][Scale 1],-0.014754,-0.040782,-0.118534,0.653233,-0.053107,0.042315,0.638260,0.536532,1.000000,0.575772,0.227962,0.122547,0.234320,0.389507,0.192811
How helpful do you find the following KPIs for SCRUM p

## Analyse participant data

In [7]:
part_data = df.copy()

In [8]:
# analyse experience question
exp_stats = part_data[experience_column].value_counts()
exp_stats_percentage = part_data[experience_column].value_counts(normalize=True) * 100

print(exp_stats)
print(exp_stats_percentage)

How many years of expirience do you have with SCRUM?
4+ years            26
2-4 years           13
1-2 years            7
less than 1 year     3
Name: count, dtype: int64
How many years of expirience do you have with SCRUM?
4+ years            53.061224
2-4 years           26.530612
1-2 years           14.285714
less than 1 year     6.122449
Name: proportion, dtype: float64


In [9]:
roles_stats = part_data[role_columns].apply(pd.to_numeric, errors='coerce').agg(['mean', 'sum'])

roles_result = roles_stats.T
roles_result['mean'] *= 100
roles_result.columns = ['Percentage', 'Total']
roles_result

,Percentage,Total
Which SCRUM roles have you primarily taken on? [Product Owner],30.612245,15.0
Which SCRUM roles have you primarily taken on? [SCRUM team member],67.346939,33.0
Which SCRUM roles have you primarily taken on? [SCRUM master],28.571429,14.0


In [10]:
options_other_roles_count = part_data[other_role_column].apply(lambda x: 0 if pd.isna(x) else 1).agg(['sum'])['sum']

print('Amount of other roles provided:', options_other_roles_count)

options = part_data[part_data[other_role_column].notna()][other_role_column].values.tolist()
print('Other roles: ', options)

Amount of other roles provided: 5
Other roles:  ['Beobachter', 'Developer', 'product manager', 'Trainer pspo1', 'SCRUM-adjacent colleague observing product development process output']


## Analyse KPI data

### KPIs and ratings

This section will evaluate the surveys part about KPIs and their helpfulness

In [11]:
kpi_eval = df.copy()

First the mean, min and max rating of each KPI is computed:

In [12]:
# Compute table
cleaned_kpi_statistics = kpi_eval[kpi_columns].apply(pd.to_numeric, errors='coerce').agg(['mean', 'min', 'max'])

rename_dict = {col: extract_KPI_name(col) for col in kpi_columns}

cleaned_kpi_statistics.rename(columns=rename_dict, inplace=True)
cleaned_kpi_statistics.T.sort_values(by='mean', ascending=False)

,mean,min,max
[Capacity (How many Story Points can a Team theoretically do in a Sprint?)],3.959184,1.0,5.0
[Amount of done Story Points],3.755102,1.0,5.0
[Commitment (Story Points planned for a Sprint)],3.714286,1.0,5.0
[Velocity (Planned Story Points / Done Story Points)],3.714286,1.0,5.0
[Open Story points (How many story points are not done at the end of a Sprint and what state are they in?)],3.632653,1.0,5.0
[Blocker Tickets (Number of 'emergency' tickets that got added during a Sprint)],3.204082,1.0,5.0
[Overplanning ratio (Planned Story Points / Capacity)],3.040816,1.0,5.0
[Additional Tickets (Tickets that got added unexpectedly during the Sprint)],2.897959,1.0,5.0
[Number of done Tickets],2.857143,1.0,5.0
[Number of planned Tickets],2.816327,1.0,5.0


Secondly it should be computed where the KPI is used by the participants or not

In [13]:
kpi_use_statistics = kpi_eval[use_kpi_columns].apply(pd.to_numeric, errors='coerce').agg(['mean', 'sum'])

rename_kpis_use_names = {col: extract_KPI_name(col) for col in use_kpi_columns}

kpi_use_statistics.rename(columns=rename_kpis_use_names, inplace=True)

kpi_use_result = kpi_use_statistics.T
kpi_use_result['mean'] *= 100
kpi_use_result.columns = ['Percentage (Yes)', 'Total (Yes)']
kpi_use_result

,Percentage (Yes),Total (Yes)
[Capacity (How many Story Points can a Team theoretically do in a Sprint?)],87.755102,43.0
[Number of planned Tickets],59.183673,29.0
[Number of done Tickets],63.265306,31.0
[Commitment (Story Points planned for a Sprint)],87.755102,43.0
[Overplanning ratio (Planned Story Points / Capacity)],53.061224,26.0
[Amount of done Story Points],85.714286,42.0
[Velocity (Planned Story Points / Done Story Points)],79.591837,39.0
[Blocker Tickets (Number of 'emergency' tickets that got added during a Sprint)],48.979592,24.0
[Blocker Tickets done (Number of 'emergency' tickets that got done in the Sprint)],42.857143,21.0
[Additional Tickets (Tickets that got added unexpectedly during the Sprint)],51.020408,25.0
